In [7]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 0s (901 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [15]:
from IPython.display import clear_output
!ollama pull llama2
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏  60 KB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏ 7.1 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  39 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  66 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  81 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 109 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 127 MB/3.8 GB                  pulling ma

In [10]:
!pip install langchain_community
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader('https://python.langchain.com/docs/expression_language/')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

splitDocs = text_splitter.split_documents(docs)

  Using cached langchain_community-0.3.7-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [16]:
!pip install chromadb
# 2. Convert documents to Embeddings and store them
from langchain_community.vectorstores import Chroma
#!pip install langchain_ollama

from langchain_ollama import embeddings
from langchain_community.vectorstores.faiss import FAISS

vectorStore = Chroma.from_documents(
    documents=splitDocs,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
)

#vectorStore = FAISS.from_documents(splitDocs, embedding=embeddings)
retriever = vectorStore.as_retriever()

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

!pip install langchain-ollama

from langchain_ollama.llms import OllamaLLM

MODEL = "llama2"
model = OllamaLLM(model=MODEL)
prompt = ChatPromptTemplate.from_template("""
Answer the user's question.
Context: {context}
Question: {input}
""")

# chain = prompt | model
document_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt
)

retriever = vectorStore.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [18]:
response = retrieval_chain.invoke({
    "input": "What is LCEL?",
})

print(response)

{'input': 'What is LCEL?', 'context': [Document(metadata={'description': 'This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.', 'language': 'en', 'source': 'https://python.langchain.com/docs/expression_language/', 'title': 'Conceptual guide | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain'}, page_content='LangChain Expression Language (LCEL): A syntax for orchestrating LangChain components. Most useful for simpler applications.\nDocument loaders: Load a source as a list of documents.'), Document(metadata={'description': 'This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.', 'language': 'en', 'source': 'https://python.langchain.com/docs/expression_language/', 'title': 'Conceptual guide | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain'}, page_content='with_types: Method to overwrite the input and output types of a runnable. Useful when working with complex LCEL chains and deploying with 